<a href="https://colab.research.google.com/github/advaithsujith/Mount_Rainier_Hike_Model/blob/main/mountain_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Introduction**

Mount Rainier, an iconic peak in the Pacific Northwest, presents an enduring challenge for climbers. Scaling its heights requires courage, skill, and determination, and success is often shaped by the capricious forces of nature.

In this project, I developed a TensorFlow-based model to predict the success or failure of climbers on Mount Rainier. Using historical data encompassing weather conditions, climber information, and environmental factors, i hope to provide valuable insights for climbers and outdoor organizations to enhance safety and decision-making. This project's goal is to make Mount Rainier ascents more predictable and safer for all adventurers.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# **Importing both the Datasets**

In [ ]:
Weather_df = pd.read_csv('Rainier_Weather.csv')
Climbing_df = pd.read_csv('climbing_statistics.csv')

lets take a look at the datasets.

In [ ]:
Weather_df.head()

,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,12/31/2015,13.845000,19.062917,21.870833,21.977792,62.325833,84.915292
1,12/30/2015,13.822917,14.631208,18.493833,3.540542,121.505417,86.192833
2,12/29/2015,13.834583,6.614292,34.072917,0.000000,130.291667,85.100917
3,12/28/2015,13.710417,8.687042,70.557917,0.000000,164.683750,86.241250
4,12/27/2015,13.362500,14.140417,95.754167,0.000000,268.479167,31.090708


In [ ]:
Climbing_df.head()

,Date,Route,Attempted,Succeeded,Success Percentage
0,11/27/2015,Disappointment Cleaver,2,0,0.0
1,11/21/2015,Disappointment Cleaver,3,0,0.0
2,10/15/2015,Disappointment Cleaver,2,0,0.0
3,10/13/2015,Little Tahoma,8,0,0.0
4,10/9/2015,Disappointment Cleaver,2,0,0.0


As you can see, the date on both the tables can be considerd as a foreign key between both the datasets.


therefore, **I merged the tables on the Date**

In [ ]:
df = pd.merge(Weather_df, Climbing_df, on='Date', how='inner')
df.head()

,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG,Route,Attempted,Succeeded,Success Percentage
0,11/27/2015,13.643750,26.321667,19.715000,27.839583,68.004167,88.496250,Disappointment Cleaver,2,0,0.0
1,11/21/2015,13.749583,31.300000,21.690708,2.245833,117.549667,93.660417,Disappointment Cleaver,3,0,0.0
2,10/15/2015,13.461250,46.447917,27.211250,17.163625,259.121375,138.387000,Disappointment Cleaver,2,0,0.0
3,10/13/2015,13.532083,40.979583,28.335708,19.591167,279.779167,176.382667,Little Tahoma,8,0,0.0
4,10/9/2015,13.216250,38.260417,74.329167,65.138333,264.687500,27.791292,Disappointment Cleaver,2,0,0.0


The Dataframe is already looking much nicer, but we still have some more data architecting to do.

#**Starting with The Date.**



Although the date was incredibly helpful in merging the datasets together, the only feature we need from it is the season in which the date is in. Therefore, i made a function to change the date into the respective season of the year

In [ ]:
def date_to_season(date):
    month = int(date.split('/')[0])
    if 3 <= month <= 5:
        return 'Spring'

    elif 6 <= month <= 8:
        return 'Summer'

    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

df['Season'] = df['Date'].apply(date_to_season)

df = df.drop('Date', axis=1)



Lets shuffle up the dataframe and take a look at it.

In [ ]:
df=df.sample(frac=1)
df.head()

,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG,Route,Attempted,Succeeded,Success Percentage,Season
1407,13.502083,46.228750,29.615417,19.002875,21.640417,344.787375,Disappointment Cleaver,2,0,0.000000,Summer
859,13.467500,45.185417,55.228750,16.932167,22.452917,349.924958,Disappointment Cleaver,2,0,0.000000,Summer
506,13.526667,33.303333,52.500000,7.444625,191.560875,278.793583,Kautz Glacier,3,0,0.000000,Summer
1165,13.542500,38.033750,43.064583,5.631583,148.757292,351.312500,Kautz Glacier,12,10,0.833333,Summer
474,13.460833,48.886667,27.804167,13.758083,270.736667,326.978542,glacier only - no summit attempt,2,0,0.000000,Summer


And there we go! now we have a season feature that can help the model work better.

# **The Battery Voltage Average**
The Battery Voltage Average is an intresting feature.

It is a measurement of the average battery voltage of a weather monitoring or data logging system used in mountain or remote areas. Such systems often rely on batteries to power sensors, data loggers, and communication equipment.

But when the mean of the battery voltage avg was taken, it is seen that it barely changes and averages at 13.5 volts

In [ ]:
print(df['Battery Voltage AVG'].mean())

13.502638416997362


Therefore I Decided to drop it from the Dataframe

In [ ]:
df=df.drop('Battery Voltage AVG',axis='columns')

# **The Routes**
As seen below, there are a bunch of different routes a person can take when traveling up Mount Rainier. Lets encode them to make them easier to fit in the model

In [ ]:
df["Route"].unique()

array(['Disappointment Cleaver', 'Kautz Glacier',
       'glacier only - no summit attempt', 'Little Tahoma',
       'Emmons-Winthrop', 'Liberty RIngraham Directge', 'Ingraham Direct',
       'Ptarmigan RIngraham Directge', 'Tahoma Cleaver', 'Fuhrers Finger',
       'Gibralter Ledges', 'Mowich Face', "Fuhrer's Finger",
       'Curtis RIngraham Directge', 'Wilson Headwall', 'Unknown',
       'Tahoma Glacier', 'Kautz Cleaver', 'Success Cleaver',
       'Gibralter Chute', 'Sunset RIngraham Directge',
       'Nisqually Glacier'], dtype=object)

In [ ]:
label_encoder = LabelEncoder()
df['Route'] = label_encoder.fit_transform(df['Route'])
df['Season']=label_encoder.fit_transform(df['Season'])
df.head()

,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG,Route,Attempted,Succeeded,Success Percentage,Season
1407,46.228750,29.615417,19.002875,21.640417,344.787375,1,2,0,0.000000,2
859,45.185417,55.228750,16.932167,22.452917,349.924958,1,2,0,0.000000,2
506,33.303333,52.500000,7.444625,191.560875,278.793583,9,3,0,0.000000,2
1165,38.033750,43.064583,5.631583,148.757292,351.312500,9,12,10,0.833333,2
474,48.886667,27.804167,13.758083,270.736667,326.978542,21,2,0,0.000000,2


# **Succeeded or Failed to Travel up the mountain**
I decided to use the success Percentage as the y_train.

In [ ]:
df.head()

,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG,Route,Attempted,Succeeded,Success Percentage,Season
1407,46.228750,29.615417,19.002875,21.640417,344.787375,1,2,0,0.000000,2
859,45.185417,55.228750,16.932167,22.452917,349.924958,1,2,0,0.000000,2
506,33.303333,52.500000,7.444625,191.560875,278.793583,9,3,0,0.000000,2
1165,38.033750,43.064583,5.631583,148.757292,351.312500,9,12,10,0.833333,2
474,48.886667,27.804167,13.758083,270.736667,326.978542,21,2,0,0.000000,2


# **Splitting the Data**
i split the data into train, and test

In [ ]:
train,test=np.split(df.sample(frac=1),[int(0.8*len(df))])

# **Scaling Data**
for scaling my data, i decided to make a function called Scale_Dataset, and used Sklearns StandardScaler to scale it. I then returned x_train and y_train as tuples.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def Scale_Dataset(dataframe):
  y_train=dataframe['Success Percentage']
  x_train=dataframe.drop('Success Percentage',axis='columns')

  scaler=StandardScaler()

  # Fit and transform the scaler on your data
  x_train = scaler.fit_transform(x_train)

  return x_train,y_train

In [ ]:
x_train,y_train=Scale_Dataset(df)

# **Making the Model**
I used keras to make a neural network to predict weather the person would succeed or fail.

And after a bit of trial and error, i decided to go with a neural network of 4 layers, the first 3 having rectified linear unit as thier activation function, and the last one being sigmoid. I then compiled the neural netowork with an optimizer of adam, and calculated my loss using binary crossentropy.

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(x_train.shape[1],), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
    metrics=['accuracy']
)


I then fit the model with 150 epochs, again by trial and error.

In [ ]:
model.fit(x_train,y_train,epochs=150)

Epoch 1/150
60/60 [==============================] - 1s 2ms/step - loss: 0.3782 - accuracy: 0.5858
Epoch 2/150
60/60 [==============================] - 0s 2ms/step - loss: 0.1819 - accuracy: 0.7388
Epoch 3/150
60/60 [==============================] - 0s 2ms/step - loss: 0.0895 - accuracy: 0.7879
Epoch 4/150
60/60 [==============================] - 0s 2ms/step - loss: 0.0624 - accuracy: 0.7979
Epoch 5/150
60/60 [==============================] - 0s 2ms/step - loss: 0.0526 - accuracy: 0.8011
Epoch 6/150
60/60 [==============================] - 0s 3ms/step - loss: 0.0450 - accuracy: 0.8005
Epoch 7/150
60/60 [==============================] - 0s 3ms/step - loss: 0.0396 - accuracy: 0.8016
Epoch 8/150
60/60 [==============================] - 0s 3ms/step - loss: 0.0361 - accuracy: 0.8011
Epoch 9/150
60/60 [==============================] - 0s 3ms/step - loss: 0.0335 - accuracy: 0.8016
Epoch 10/150
60/60 [==============================] - 0s 3ms/step - loss: 0.0319 - accuracy: 0.8016
Epoch 11/

I ended up with an accuracy of **80.016%**, which is decent for the size of the dataset i had to work with. But to make sure i had'nt overfit, i had to obviously run my model on the test data as well, and achieved an accuracy of **80.21%**!

In [ ]:
x_test,y_test=Scale_Dataset(test)
model.evaluate(x_test,y_test)

12/12 [==============================] - 0s 2ms/step - loss: 0.0836 - accuracy: 0.7942


[0.08355433493852615, 0.7941952347755432]